# Il verme è tratto
Cercherò di creare una trasposizione di questo gioco da tavola, se mai ci riesca.

## Idee
Quello che potrei sviluppare

* Per iniziare potrei gestire il gioco fra persone
* Poi potrei integrare un pochetto di grafica (guizero?)
* Poi potrei inserire intelligenza artificiale (difficile, basata sulle probabilità)
* Infine potrei tentare di integrare il gioco in rete

## Regolamento
Documento contenente le regole

In [70]:
%%HTML
<iframe src=./regole.pdf width=100% height=350></iframe>

## Svolgimento
In questa sezione cercherò di spiegare i passaggi della partita per comprendere meglio l'algoritmo da adottare

1. Si decide il numero di giocatori (da 2 a 7) e li si ordina per età dal più giovane al più vecchio (in origine si gioca in senso orario a partire dal più giovane)
2. Si dispongono le tessere sul tavolo a formare una griglia in ordine numerico (da 21 a 36)
3. Il primo giocatore lancia i dadi:
    1. lancia 8 dadi e sceglie quale serie tenere con lo stesso numero in comune e può rilanciare i dadi rimanenti fino a quando:
        * ottiene un numero maggiore o uguale a una tessera disponibie dalla griglia del tavolo
        * ottiene un numero uguale ad una tessera in cima ad una pila di un altro giocatore
        * non ci sono più serie con numeri in comune non precedentemente trattenuti (non si possono scegliere piu volte serie con lo stesso numero in comune)
        * non si è almeno trattenuto una volta una serie con il simbolo del verme
        * non ci sono più dadi da rilanciare
    3. il giocatore prende una una tessera e la mette in cima alla sua pila se:
        * ha totalizzato un numero maggiore o uguale ad una delle tessere disposte sulla griglia, prende una di queste
        * ha totalizzato un numero uguale a una delle tessere in cima alla pila di un altro giocatore, prende questa
    4. in tutti gli altri casi ha sballato e quindi, se possiede delle tessere, dovrà:
        * restituire quella in cima alla sua pila reinserendola nella griglia
        * se nella griglia ci sono ancora tessere dal valore più alto di quella restituita bisognerà capovolgere (e quindi rendere indisponibile) la tessera dal valore più alto presente nella griglia
4. Si passa al giocatore seguente
5. Il gioco ha fine quando non ci sono più tessere sottraibili dalla griglia
6. Vince il giocatore che ha totalizzato più vermi (ogni tessera oltre al valore numerico ha anche un punteggio rappresentato da dei vermi, da uno a quattro)

## Codice

In [71]:
import doctest, random

In [72]:
class Giocatore():
    """
    Questa classe rappresenta un giocatore
    
    Istanziando il giocatore bisogna specificare il nome e l'età (facoltativa)
    >>> g = Giocatore('aldo', 14)
    >>> g.nome  #il nome del giocatore
    'aldo'
    >>> g.eta   #la sua età
    14
    
    Senza età viene assegnata un età casuale da 0 a 100
    >>> g = Giocatore('rino')
    >>> isinstance(g.eta, int) and 0 <= g.eta <= 100
    True
    
    Età non valida: ValueError
    >>> g = Giocatore('rinco', 'asda')
    Traceback (most recent call last):
    ...
    ValueError: invalid literal for int() with base 10: 'asda'
    
    Età valida
    >>> g = Giocatore('rinco', '134')
    >>> g.eta
    134
    """
    def __init__(self, nome, eta=None):
        self.nome = nome
        if eta is None: self.eta = random.randint(0,100) #età casuale
        else: self.eta = int(eta)

In [73]:
class Tessera():
    """
    Questa classe rappresenta una tessera
    
    >>> t = Tessera(21, 1)
    >>> t.valore, t.punti
    (21, 1)
    """
    def __init__(self, valore, punti):
        self.valore = valore
        self.punti = punti
    
    def __repr__(self):
        return "Tessera({}, {})".format(self.valore, self.punti)

In [74]:
class Dado():
    """
    Questa classe rappresenta un dado
    
    >>> d = Dado(6)
    >>> d.faccev
    (1, 2, 3, 4, 5, 6)
    >>> l = d.lancia(100)
    >>> all([n in [1,2,3,4,5,6] for n in l])
    True
    """
    def __init__(self, facce):
        self.faccev = tuple(range(1, facce+1))
        
    def lancia(self, numero=1):
        """Lancia il dado n volte"""
        #return random.choice(self.faccev)
        return [random.choice(self.faccev) for n in range(numero)]           

In [75]:
class Tavolo():
    """
    Questa classe serve a gestire gli spostamenti sul tavolo
    
    >>> g1, g2 = [Giocatore(g) for g in ['a', 'b']]
    >>> tav = Tavolo([g1, g2])
    >>> isinstance(tav.giocatori[0], Giocatore)
    True
    """
    def __init__(self, giocatori):
        self.giocatori = giocatori
        self.griglia = self.inizializza_griglia_tessere()
    
    def inizializza_griglia_tessere(self):
        """Questa funzione genera una lista di tessere"""
        tessere = []
        p = 1
        for v in range (21, 37):
            tessere.append(Tessera(v, int(p)))
            p += 0.25
        return tessere

### doctest

In [76]:
doctest.testmod()

TestResults(failed=0, attempted=17)

### codice inutilizzato

In [ ]:
def inizializza_griglia_tessere(car="tessere_gif/", est=".gif"):
    """
    Questa funzione genera una lista di tessere
    
    >>> ts = genera_tessere()
    >>> ts
    [(21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 2), (27, 2), (28, 2), (29, 3), (30, 3), (31, 3), (32, 3), (33, 4), (34, 4), (35, 4), (36, 4)]
    >>> from os.path import isfile
    >>> all([isfile(t.immagine) for t in ts])
    True
    """
    tessere = []
    p = 1
    for v in range (21, 37):
        path = car + str(v) + est
        tessere.append(Tessera(v, int(p), path))
        p += 0.25
    return tessere

In [ ]:
#metto i doctest dentro un altra funzione alla fine, se non li voglio nelle docstring
#di ogni singola classe e funzione potrei fare così e metterli a parte
def test_giocatore():
    """
    Questa classe rappresenta un giocatore
    
    Istanziando il giocatore bisogna specificare il nome e l'età (facoltativa)
    >>> g = Giocatore('aldo', 14)
    >>> g.nome  #il nome del giocatore
    'aldo'
    >>> g.eta   #la sua età
    14
    
    Senza età viene assegnata un età casuale da 0 a 100
    >>> g = Giocatore('rino')
    >>> isinstance(g.eta, int) and 0 <= g.eta <= 100
    True
    
    Età non valida sempre età casuale
    >>> g = Giocatore('rinco', 'asda')
    >>> isinstance(g.eta, int) and 0 <= g.eta <= 100
    True
    
    Età valida
    >>> g = Giocatore('rinco', '134')
    >>> g.eta
    134
    """
    pass
doctest.testmod()